In [14]:
from datasets import load_dataset

dataset = load_dataset("coseal/Magicoder-Evol-Instruct-110K-sft")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response'],
        num_rows: 100772
    })
})


In [15]:
import pandas as pd

df = pd.DataFrame(dataset["train"])
df

,instruction,response
0,Please amend the subsequent Python script so t...,```python\n# Establish an integer list\narr = ...
1,"i've got this python code from an ocr tool, bu...",There are several issues with your code like f...
2,Create a recursive function in Java that predi...,You can achieve this using a combination of re...
3,Develop a program that uses natural language p...,This task requires writing of a significant vo...
4,I am trying to create a load testing script fo...,The correct approach would depend on the preci...
...,...,...
100767,Create a function that not only detects any ch...,"In Python, we could use the difflib module and..."
100768,Conduct a comprehensive exploratory data analy...,"Sorry, as an AI developed by Open AI, I can't ..."
100769,"I used OCR to get this Python code, but it's f...","Sure, I can help with that. Let me reformat th..."
100770,Write a JavaScript function that will take an ...,"Sure, I'll provide you with a simple example o..."


In [16]:
from metagpt.utils.common import OutputParser

def is_python_code(row):
    # instruction = row.instruction
    response = row.response

    try:
        code = OutputParser.parse_code(response, "python")
        if len(code) > 10:
            return True
    except:
        pass
    return False

df["ispy"] = df.apply(is_python_code, axis=1)
xdf = df[df.ispy][["instruction","response"]].copy().rename(columns={"instruction": "input", "response": "output"})
xdf

2024-03-26 19:23:32.328 | INFO     | metagpt.const:get_metagpt_package_root:29 - Package root set to /Users/deryzhou/Downloads/StreamChatPlayground/notebooks


,input,output
0,Please amend the subsequent Python script so t...,```python\n# Establish an integer list\narr = ...
1,"i've got this python code from an ocr tool, bu...",There are several issues with your code like f...
3,Develop a program that uses natural language p...,This task requires writing of a significant vo...
8,Develop a method that not only incorporates a ...,You can implement this task in Python:\n\n```p...
9,Write a Python program that takes a list and a...,Here is a Python program that does just that:\...
...,...,...
100763,Modify and optimize the given erroneous code t...,This erroneous code uses the bubble sort algor...
100764,Utilize the BeautifulSoup package to orchestra...,You can extract the title tag content from the...
100765,Implement a function that can insert multiple ...,Here is the Python implementation:\n\n```pytho...
100767,Create a function that not only detects any ch...,"In Python, we could use the difflib module and..."


In [17]:
xdf.to_json("magicoder.jsonl", orient='records', lines=True)